## Title: Customer Churn Prediction (MySQL + Logistic Regression)

Objective: Train a logistic regression model to predict customer churn, write predicted probabilities back to MySQL, and export a CSV for visualization (e.g., Tableau).

### 0. Environment & Configuration

"Set database credentials in a .env file (not tracked in git). See .env.example for structure."

In [20]:
from dotenv import load_dotenv
load_dotenv(override=True)  # IMPORTANT: allows replacing a previously-set value
import os

load_dotenv()  # reads .env if present

MYSQL_HOST = os.getenv("MYSQL_HOST", "127.0.0.1")
MYSQL_PORT = int(os.getenv("MYSQL_PORT", "3306"))
MYSQL_USER = os.getenv("MYSQL_USER", "root")
MYSQL_PASSWORD = os.getenv("MYSQL_PASSWORD")  # keep secrets out of notebook
MYSQL_DB = os.getenv("MYSQL_DB", "churn_project")

In [ ]:
import pandas as pd
import mysql.connector as mc
import os
from dotenv import load_dotenv

load_dotenv(override=True)

QUERY = """
SELECT
  customerID AS id,
  is_monthly,
  auto_pay,
  CASE WHEN tenure < 12 THEN 1 ELSE 0 END AS short_tenure,
  churn
FROM customers_clean;
"""

conn = mc.connect(
    host=os.getenv("MYSQL_HOST","127.0.0.1"),
    port=int(os.getenv("MYSQL_PORT","3306")),
    user=os.getenv("MYSQL_USER"),
    password=os.getenv("MYSQL_PASSWORD"),
    database=os.getenv("MYSQL_DB","churn_project")
)
df = pd.read_sql(QUERY, conn)
conn.close()

df.head(10)

/tmp/ipykernel_66269/1861794141.py:25: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(QUERY, conn)


DatabaseError: Execution failed on sql '
SELECT
  customerID AS id,
  is_monthly,
  auto_pay,
  CASE WHEN tenure < 12 THEN 1 ELSE 0 END AS short_tenure,
  churn
FROM customers_clean;
': 1146 (42S02): Table 'churn_project.customers_clean' doesn't exist

In [19]:
from dotenv import load_dotenv; load_dotenv(override=True)
import os, mysql.connector as mc

host = os.getenv("MYSQL_HOST", "127.0.0.1")
port = int(os.getenv("MYSQL_PORT", "3306"))
print("Connecting to", host, port)

conn = mc.connect(
    host=host, port=port,
    user=os.getenv("MYSQL_USER"),
    password=os.getenv("MYSQL_PASSWORD"),
    database=os.getenv("MYSQL_DB")
)
cur = conn.cursor()
cur.execute("SELECT DATABASE()")
print("DB:", cur.fetchone())
cur.close(); conn.close()

Connecting to 127.0.0.1 3306
DB: ('churn_project',)
